<a href="https://colab.research.google.com/github/Robbysaidiii/Scraping_RobbySaidi_mc211d5y2136/blob/main/kalsifijkasi_gambar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install tensorflowjs

## **Import Packages**

In [13]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img
from tensorflow.keras.optimizers import Adam,RMSprop,SGD
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os
import matplotlib.pyplot as plt
from PIL import ImageEnhance, Image
from collections import Counter
from tqdm import tqdm


## **Import Dataset**

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
zip_path = '/content/drive/MyDrive/dataset_food.zip'


In [16]:
!unzip -q "/content/drive/MyDrive/dataset_food.zip" -d "/content/"

replace /content/dataset_food/test/bakso/bakso-22052.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
A


# **resolusi gambar**

In [17]:

dataset_path = '/content/dataset_food/train'

# Simpan semua resolusi
resolutions = []

for root, dirs, files in os.walk(dataset_path):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png')):
            try:
                img_path = os.path.join(root, file)
                img = Image.open(img_path)
                resolutions.append(img.size)  # (width, height)
            except:
                continue

# Hitung dan tampilkan ukuran yang paling umum
res_counter = Counter(resolutions)
print("Top 10 resolusi gambar yang ditemukan:")
print(res_counter.most_common(10))

# Total variasi resolusi
print(f"\nTotal variasi resolusi unik: {len(set(resolutions))}")


Top 10 resolusi gambar yang ditemukan:
[((299, 299), 6597), ((200, 200), 862), ((1200, 1200), 27), ((720, 720), 21), ((450, 450), 20), ((465, 465), 19), ((768, 768), 19), ((400, 400), 19), ((361, 361), 18), ((353, 353), 18)]

Total variasi resolusi unik: 275


Ini membuktikan bahwa dataset belum melalui preprocessing sebelumnya dan terdiri dari gambar-gambar asli.

# **Preprocessing Data**

In [18]:
# Set path dan parameter
img_height, img_width, img_channels = 224, 224, 3
batch_size = 32

train_dir = '/content/dataset_food/train'
valid_dir = '/content/dataset_food/valid'
test_dir  = '/content/dataset_food/test'

# Image Augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   horizontal_flip=True,
                                   zoom_range=0.2,
                                   rotation_range=10)

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

# Load data
train_data = train_datagen.flow_from_directory(train_dir, target_size=(img_width, img_height),
                                               batch_size=batch_size, class_mode='categorical')

valid_data = valid_datagen.flow_from_directory(valid_dir, target_size=(img_width, img_height),
                                               batch_size=batch_size, class_mode='categorical')

test_data = test_datagen.flow_from_directory(test_dir, target_size=(img_width, img_height),
                                             batch_size=batch_size, class_mode='categorical')

num_classes = train_data.num_classes


Found 9000 images belonging to 5 classes.
Found 902 images belonging to 5 classes.
Found 432 images belonging to 5 classes.


- train mempunyai  3949 gabar
- valid mempnyai 902 gambar
- test mempunyai 432 gambr
- ▶ total semua nya  5283 gambar tidak memenuhi syarat penlaian yang harus 10.000 data. maka dari itu Train saya akan augmentasi

# **Augmentasi Train**

In [19]:
# Konfigurasi
source_dir = '/content/dataset_food/train'
target_per_class = 1800  # Target gambar per kelas

# ImageDataGenerator (augmentasi standar)
augmentor = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Augmentasi manual tambahan
def apply_high_contrast(img):
    enhancer = ImageEnhance.Contrast(img)
    return enhancer.enhance(2)

def apply_low_contrast(img):
    enhancer = ImageEnhance.Contrast(img)
    return enhancer.enhance(0.5)

def apply_darkened(img):
    enhancer = ImageEnhance.Brightness(img)
    return enhancer.enhance(0.5)

def apply_cropped(img, crop_size=(200, 200)):
    width, height = img.size
    left = (width - crop_size[0]) // 2
    top = (height - crop_size[1]) // 2
    right = (width + crop_size[0]) // 2
    bottom = (height + crop_size[1]) // 2
    return img.crop((left, top, right, bottom))

def apply_inverted_rotation(img):
    return img.rotate(180)

# Mulai augmentasi per kelas
for class_dir in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_dir)
    images = os.listdir(class_path)
    current_class_count = len(images)

    print(f"[{class_dir}] Jumlah awal: {current_class_count}")

    i = 0
    while current_class_count < target_per_class:
        img_path = os.path.join(class_path, images[i % len(images)])
        img = load_img(img_path)
        x = img_to_array(img)
        x = x.reshape((1,) + x.shape)

        # Augmentasi standar
        aug_iter = augmentor.flow(x, batch_size=1)
        aug_img = next(aug_iter)[0].astype(np.uint8)
        img_aug = array_to_img(aug_img)
        save_path = os.path.join(class_path, f'aug_{i}_{images[i % len(images)]}')
        img_aug.save(save_path)
        current_class_count += 1

        # Augmentasi manual
        img_pil = Image.open(img_path)
        manual_augs = [
            (apply_high_contrast, 'high_contrast'),
            (apply_low_contrast, 'low_contrast'),
            (apply_darkened, 'darkened'),
            (apply_cropped, 'cropped'),
            (apply_inverted_rotation, 'inverted_rotated')
        ]

        for func, prefix in manual_augs:
            if current_class_count >= target_per_class:
                break
            aug_img = func(img_pil)
            aug_path = os.path.join(class_path, f'{prefix}_{i}_{images[i % len(images)]}')
            aug_img.save(aug_path)
            current_class_count += 1

        i += 1
        if i % 100 == 0:
            print(f"[{class_dir}] Augmented {i} gambar. Sekarang: {current_class_count}")

    print(f"[{class_dir}] Augmentasi selesai. Total gambar: {current_class_count}")

print("Augmentasi selesai untuk semua kelas.")


[gado] Jumlah awal: 1800
[gado] Augmentasi selesai. Total gambar: 1800
[sate] Jumlah awal: 1800
[sate] Augmentasi selesai. Total gambar: 1800
[gudeg] Jumlah awal: 1800
[gudeg] Augmentasi selesai. Total gambar: 1800
[bakso] Jumlah awal: 1800
[bakso] Augmentasi selesai. Total gambar: 1800
[rendang] Jumlah awal: 1800
[rendang] Augmentasi selesai. Total gambar: 1800
Augmentasi selesai untuk semua kelas.


- Train Data: Augmentasi diterapkan di sini untuk meningkatkan variasi data, agar model belajar dari berbagai variasi gambar. Ini membantu mencegah overfitting.

- Validation Data: Tidak dilakukan augmentasi pada data ini. Validasi digunakan untuk mengukur performa model pada data yang belum pernah dilihat oleh model sebelumnya.

- Test Data: Sama seperti validation, test data juga tidak diubah, untuk mengevaluasi kemampuan model dalam mengenali gambar yang tidak terlibat dalam pelatihan.

In [20]:
def count_images_in_directory(directory):
    total_images = 0
    for class_dir in os.listdir(directory):
        class_path = os.path.join(directory, class_dir)
        if os.path.isdir(class_path):
            total_images += len(os.listdir(class_path))
    return total_images

train_dir = '/content/dataset_food/train'
valid_dir = '/content/dataset_food/valid'
test_dir = '/content/dataset_food/test'

total_train = count_images_in_directory(train_dir)
total_valid = count_images_in_directory(valid_dir)
total_test = count_images_in_directory(test_dir)

print(f"Total gambar di TRAIN SET: {total_train}")
print(f"Total gambar di VALIDATION SET: {total_valid}")
print(f"Total gambar di TEST SET: {total_test}")
print(f"Total keseluruhan: {total_train + total_valid + total_test}")


Total gambar di TRAIN SET: 9000
Total gambar di VALIDATION SET: 902
Total gambar di TEST SET: 432
Total keseluruhan: 10334


seperti yang sudah saya jelaskan diatas data gambar yang ditambhkan dengan augmentasi adalah data train. total keseluruha gaar data adalah 10334

# **Membangun Arsitektur CNN**

In [21]:

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(img_width, img_height, 3)),
    BatchNormalization(),
    Dropout(0.25),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.25),
    MaxPooling2D(2, 2),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Dropout(0.25),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 224, 224, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 224, 224, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 56, 56, 256)    │       147,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 56, 56, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 56, 56, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 200704)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │   102,760,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 102,932,037 (392.65 MB)

 Trainable params: 102,931,333 (392.65 MB)

 Non-trainable params: 704 (2.75 KB)

Model ini terdiri dari beberapa lapisan utama. Pertama, tiga buah Conv2D (konvolusi) dengan jumlah filter berturut-turut 32, 64, dan 128 digunakan untuk mengekstrak fitur dari gambar. Setelah masing-masing Conv2D, terdapat MaxPooling2D, yang bertugas mengurangi dimensi spasial (tinggi dan lebar) sehingga mempercepat komputasi dan membantu mengurangi overfitting. Setelah itu, lapisan Flatten digunakan untuk mengubah keluaran 3D dari konvolusi menjadi bentuk 1D agar dapat diproses oleh lapisan dense (fully connected). Kemudian, terdapat Dense layer dengan 128 neuron yang berfungsi untuk pembelajaran klasifikasi, diikuti Dropout untuk mencegah overfitting. Lapisan terakhir adalah Dense layer dengan 5 neuron, sesuai jumlah kelas, yang berfungsi sebagai output layer. Model ini memiliki total 11.169.605 parameter, semuanya dapat dilatih (trainable). Sebagian besar parameter berasal dari dense layer karena ukurannya besar setelah data diflatten.

# **Training Model + Callback**

In [22]:
earlystop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint("best_model.h5", save_best_only=True)

history = model.fit(train_data,
                    epochs=50,
                    validation_data=valid_data,
                    callbacks=[earlystop, checkpoint],
                    verbose=1)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
199/282 ━━━━━━━━━━━━━━━━━━━━ 16:19 12s/step - accuracy: 0.3083 - loss: 43.0123

KeyboardInterrupt: 

# **Evaluasi Model**

In [ ]:
test_loss, test_acc = model.evaluate(test_data)
print(f'\nTest Accuracy: {test_acc*100:.2f}%')

# **Visualisasi Akurasi dan Loss**

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend(); plt.title('Model Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend(); plt.title('Model Loss')
plt.show()


# **Simpan Model dalam Berbagai Format**

In [ ]:
# SavedModel
model.save("saved_model/")

# TF-Lite
converter = tf.lite.TFLiteConverter.from_saved_model("saved_model/")
tflite_model = converter.convert()
with open("model.tflite", "wb") as f:
    f.write(tflite_model)

# TFJS
!tensorflowjs_converter --input_format=tf_saved_model saved_model/ tfjs_model/


# **Inference**


In [ ]:
from google.colab import files
from tensorflow.keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
    path = '/content/' + fn
    img = image.load_img(path, target_size=(img_width, img_height))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array)
    class_idx = np.argmax(pred[0])
    class_labels = list(train_data.class_indices.keys())
    predicted_class = class_labels[class_idx]

    # Tampilkan hasil
    plt.imshow(img)
    plt.title(f"Prediction: {predicted_class}")
    plt.axis('off')
    plt.show()
